In [1]:
!pip install cplex
!pip install docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.3 MB 23 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 610 kB 24.9 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=50f02889cdd0f84e45059d6134c762681243d2104a7e92530e6abe8086d9e353
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import docplex.mp.model as md
from scipy.spatial import Delaunay
import random
from itertools import permutations

In [59]:
def check_solution(u,s,m,solution):
  cover=[]
  count=0
  for i in range(m):
    if solution[i]==1:
      cover.append(s[i])

#  for i in range(m):
#    for j in range(n):
#      if solution[i,j]==1 and count==i:
#        cover.append(s[i])
#        count+=1
#        break

  total=0
  for i in u:
    for j in cover:
      if i in j:
        total+=1
        break
      else:
        continue
  for i in u:
    count=0
    for j in cover:
      if i in j:
        count+=1
    #print(str(i)+': ',count)
  if len(u)==total:
    print('Solution contains all elements from U.')

  count_2=0
  for i in cover:
    for j in cover:
      count_2+=1
      if i!=j and i.issubset(j):
        print('Incorrect solution as '+str(i)+' is contained in '+str(j)+'.')
        count_2+=len(cover)*len(cover)
        break
  
  for i in u:
    count_3=0
    for j in cover:
      if i in j:
        count_3+=1
    if count_3==0:
      print('Incorrect solution as '+str(i)+' is not contained in the cover.')

  p=len(cover)
  if count_2==p*p and len(u)==total:
    print('A correct solution was found as no subset in the solution \n cover was contained within another subset.')  
#print('The number of elements in U is '+str(len(u))+'.')
#print('The number of distinct elements in the solution is '+str(total)+'.')


def is_a_subset(u,s,solution):
  cover=set()
  count=0
  for i in range(m):
    if solution[i]==1:
      cover |=s[i]
#  for i in range(m):
#    for j in range(n):
#      if solution[i,j]==1 and count==i:
#        cover |= s[i]
#        count+=1
#        break
  return u.issubset(cover)

**Set Cover Problem (NP-Complete problem)**

**Goal:** Given a set of $n$ numbers $U$, called the universe and a set $S$ of $m$ subsets of $U$ such the union of the $n$ subsets of $S$ equals $U$, i.e. $\cup_{i=1}^m S_i$ for $S_i \in S$, find the smallest collection of subsets in $S$ that cover $U$.

**Example:** Given the universe $U=\{1,2,3,4\}$ and the set $S=\{\{1,2,3\},\{1\},\{2,3\},\{3,4\}\}$, the solution would be $\{1,2,3\}$ and $\{ 3,4\}$.

Place a qubit $x_i$ on each of the $m$ subsets in $S$. (Significantly less qubits than Lucas solution which uses $n*m +m$ qubits.)

$$
H= \underbrace{ A \sum_{i=1}^m x_{i}}_{\text{Minimizes the} \\ \text{number of subsets}} + B \underbrace{\sum_{e=1}^n \left[\sum_{i=1 \\
\text{e $\in$ S_i}}^m x_{i} -1.4 \right]^2}_{\text{minimizes the number} \\ \text{of sets used per element}} + C \underbrace{\sum_{i=1}^m \left[ \sum_{j=1 \\ \text{$S_j \in S_i$}}^m len(s[i])x_i+ len(s[j])x_j  - len(s[i]) \right]^2}_{\text{Chooses largest subset $S_i \in S$, which contains another set $S_j$}}$$

We choose $1.4$ as we prefer $1$ or $2$  sets for each element, but preferably $1$.

---------

Note: Run this first segment of code as many times as does not admit an error so that the set $S$ has no empty sets or duplicate subsets.

In [88]:
n,m,A,B,C=5,5,1,10,1
u = set(random.sample(range(n), n))
s=[]
control=set()
for i in range(m-1):
  subset_size = random.randrange(m)
  subset = set(random.sample(u, subset_size))
  s+=[subset]
  control|=subset


# Add in remaining elements
rest = u - control 
print('We need still need to add ',rest,' to S in order for S to contain all elements in U.')    
if rest:
  s+=[rest]

# Take out duplicates and empty sets
for i in range(len(s)):
  for j in range(i+1,len(s)):
    if s[i]==s[j]:
      s.pop(j)

s=[i for i in s if i!=set()]

m=len(s)

print('U = ',u)
print('S = ',s)

We need still need to add  set()  to S in order for S to contain all elements in U.
U =  {0, 1, 2, 3, 4}
S =  [{2}, {0, 1, 3, 4}, {0, 1}, {1, 2}]


In [89]:
model = md.Model(name='Set_Cover',parameters={})
x = np.array(model.binary_var_list(m))
#x = np.array(model.binary_var_list(m*n)).reshape(m,n)
H1 = A*np.sum([[ x[i] ] for i in range(m)])
H2 = B*np.sum([ (np.sum([ [x[i]] for i in range(m) if e in s[i] ]) -1.4 )**2  for e in u ])
#H2 = B*np.sum([ np.sum([ [x[i][e]-0.8] for i in range(m) if e in s[i] ])**2  for e in u ])
H3 = C*np.sum([ np.sum([ [ len(s[i])*x[i]-len(s[j])*x[j] - len(s[i])  ] for j in range(m) if s[j].issubset(s[i]) ])**2 for i in range(m) ])
#H3 = C*np.sum([ (np.sum(x[:,e])-1)**2  for e in range(n) ])
H=H1+H2+H3
model.minimize(H)
solution = np.array(model.solve().get_value_list(x.flatten()))
#solution = np.array(model.solve().get_value_list(x.flatten())).reshape(m,n)
print(solution)

[0. 1. 0. 1.]


In [90]:
print('U = ',u)
print('S = ',s)
count=0
for i in range(m):
  if solution[i]==1:
    count+=1
    print('Subset '+str(i)+' in S is the solution\'s '+str(count)+' subset'+': ',s[i])

U =  {0, 1, 2, 3, 4}
S =  [{2}, {0, 1, 3, 4}, {0, 1}, {1, 2}]
Subset 1 in S is the solution's 1 subset:  {0, 1, 3, 4}
Subset 3 in S is the solution's 2 subset:  {1, 2}


In [91]:
if is_a_subset(u,s,solution):
  print('All elements from U are contained within the cover solution.')
# Check the Solution. Checks by confirming there are n distinct elements
check_solution(u,s,m,solution)

All elements from U are contained within the cover solution.
Solution contains all elements from U.
A correct solution was found as no subset in the solution 
 cover was contained within another subset.


In [ ]:
#vars(model)